# Analyse Exploratoire

### Import des modules

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns 

### Analyse Exploratoire

In [3]:
building_consumption = pd.read_csv("building_energy_benchmarking.csv")

In [4]:
# On regarde comment un batiment est défini dans ce jeu de données 
building_consumption.head()

,OSEBuildingID,DataYear,BuildingType,PrimaryPropertyType,PropertyName,Address,City,State,ZipCode,TaxParcelIdentificationNumber,...,Electricity(kWh),Electricity(kBtu),NaturalGas(therms),NaturalGas(kBtu),DefaultData,Comments,ComplianceStatus,Outlier,TotalGHGEmissions,GHGEmissionsIntensity
0,1,2016,NonResidential,Hotel,Mayflower park hotel,405 Olive way,Seattle,WA,98101.0,0659000030,...,1.156514e+06,3946027.0,12764.52930,1276453.0,False,NaN,Compliant,NaN,249.98,2.83
1,2,2016,NonResidential,Hotel,Paramount Hotel,724 Pine street,Seattle,WA,98101.0,0659000220,...,9.504252e+05,3242851.0,51450.81641,5145082.0,False,NaN,Compliant,NaN,295.86,2.86
2,3,2016,NonResidential,Hotel,5673-The Westin Seattle,1900 5th Avenue,Seattle,WA,98101.0,0659000475,...,1.451544e+07,49526664.0,14938.00000,1493800.0,False,NaN,Compliant,NaN,2089.28,2.19
3,5,2016,NonResidential,Hotel,HOTEL MAX,620 STEWART ST,Seattle,WA,98101.0,0659000640,...,8.115253e+05,2768924.0,18112.13086,1811213.0,False,NaN,Compliant,NaN,286.43,4.67
4,8,2016,NonResidential,Hotel,WARWICK SEATTLE HOTEL (ID8),401 LENORA ST,Seattle,WA,98121.0,0659000970,...,1.573449e+06,5368607.0,88039.98438,8803998.0,False,NaN,Compliant,NaN,505.01,2.88


In [5]:
# On regarde le nombre de valeurs manquantes par colonne ainsi que leur type 
building_consumption.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3376 entries, 0 to 3375
Data columns (total 46 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   OSEBuildingID                    3376 non-null   int64  
 1   DataYear                         3376 non-null   int64  
 2   BuildingType                     3376 non-null   object 
 3   PrimaryPropertyType              3376 non-null   object 
 4   PropertyName                     3376 non-null   object 
 5   Address                          3376 non-null   object 
 6   City                             3376 non-null   object 
 7   State                            3376 non-null   object 
 8   ZipCode                          3360 non-null   float64
 9   TaxParcelIdentificationNumber    3376 non-null   object 
 10  CouncilDistrictCode              3376 non-null   int64  
 11  Neighborhood                     3376 non-null   object 
 12  Latitude            

In [6]:
building_consumption["DataYear"].nunique()


1

In [7]:
building_consumption["City"].nunique()

1

In [8]:
building_consumption["State"].nunique()

1

In [9]:
building_consumption["Comments"].nunique()

0

In [10]:
building_consumption["CouncilDistrictCode"].nunique()

7

In [11]:
building_consumption["ZipCode"].nunique()

55

On peut supprimer les colonnes **DataYear**, **City**, **State** car elles ont toutes la même valeur. et **Comments** car il n'a pas de valeur.

In [12]:
building_consumption["Neighborhood"].value_counts()

Neighborhood
DOWNTOWN                  573
EAST                      453
MAGNOLIA / QUEEN ANNE     423
GREATER DUWAMISH          375
NORTHEAST                 280
LAKE UNION                251
NORTHWEST                 210
SOUTHWEST                 166
NORTH                     145
BALLARD                   126
CENTRAL                   107
SOUTHEAST                  95
DELRIDGE                   80
North                      42
Central                    27
Northwest                  11
Ballard                     7
Delridge                    4
DELRIDGE NEIGHBORHOODS      1
Name: count, dtype: int64

Est il nécessaire de garder la longitude & la latitude sachant qu'on a le voisinage?

In [13]:
building_consumption["BuildingType"].value_counts()

BuildingType
NonResidential          1460
Multifamily LR (1-4)    1018
Multifamily MR (5-9)     580
Multifamily HR (10+)     110
SPS-District K-12         98
Nonresidential COS        85
Campus                    24
Nonresidential WA          1
Name: count, dtype: int64

On peut supposer que l'on peut supprimer les lignes avec **Multifamily**

In [14]:
building_consumption_clean = building_consumption.copy()

In [15]:
building_consumption_clean = building_consumption_clean[
    ~building_consumption_clean["BuildingType"].isin(["Multifamily LR (1-4)", "Multifamily MR (5-9)", "Multifamily HR (10+)"])
]

In [16]:
building_consumption_clean = building_consumption_clean.drop(["DataYear", "City", "State", "Comments"], axis=1)


In [17]:
building_consumption_clean = building_consumption_clean.drop(["Latitude","Longitude"], axis=1)

Les données d'énergies principales sont en kBtu, on peut donc supprimer les colonnes en kWh et en therms

In [18]:
building_consumption_clean = building_consumption_clean.drop(["Electricity(kWh)", "NaturalGas(therms)"], axis=1)

In [19]:
building_consumption_clean.columns

Index(['OSEBuildingID', 'BuildingType', 'PrimaryPropertyType', 'PropertyName',
       'Address', 'ZipCode', 'TaxParcelIdentificationNumber',
       'CouncilDistrictCode', 'Neighborhood', 'YearBuilt', 'NumberofBuildings',
       'NumberofFloors', 'PropertyGFATotal', 'PropertyGFAParking',
       'PropertyGFABuilding(s)', 'ListOfAllPropertyUseTypes',
       'LargestPropertyUseType', 'LargestPropertyUseTypeGFA',
       'SecondLargestPropertyUseType', 'SecondLargestPropertyUseTypeGFA',
       'ThirdLargestPropertyUseType', 'ThirdLargestPropertyUseTypeGFA',
       'YearsENERGYSTARCertified', 'ENERGYSTARScore', 'SiteEUI(kBtu/sf)',
       'SiteEUIWN(kBtu/sf)', 'SourceEUI(kBtu/sf)', 'SourceEUIWN(kBtu/sf)',
       'SiteEnergyUse(kBtu)', 'SiteEnergyUseWN(kBtu)', 'SteamUse(kBtu)',
       'Electricity(kBtu)', 'NaturalGas(kBtu)', 'DefaultData',
       'ComplianceStatus', 'Outlier', 'TotalGHGEmissions',
       'GHGEmissionsIntensity'],
      dtype='object')

DefaultData : Données estimées ou réelles

In [20]:
building_consumption_clean["YearsENERGYSTARCertified"].value_counts()

YearsENERGYSTARCertified
2016                                12
2017                                 5
20162015                             4
2009                                 4
2013                                 4
                                    ..
20172015201420132011                 1
20152012                             1
201620152014                         1
20162015201420132012201020092008     1
20162015201020092008                 1
Name: count, Length: 64, dtype: int64

Ces données semblent peu exploitables, avec des valeurs incohérentes. On va donc supprimer cette colonne.

In [21]:
building_consumption_clean = building_consumption_clean.drop(["YearsENERGYSTARCertified"], axis=1)

In [22]:
building_consumption_clean["DefaultData"].value_counts()

DefaultData
False    1580
True       88
Name: count, dtype: int64

ComplianceStatus : Conformité réglementaire du bâtiment

In [23]:
building_consumption_clean["ComplianceStatus"].value_counts()


ComplianceStatus
Compliant                       1548
Error - Correct Default Data      88
Non-Compliant                     18
Missing Data                      14
Name: count, dtype: int64

In [24]:
building_consumption_clean[building_consumption_clean["ComplianceStatus"] == "Error - Correct Default Data"]

,OSEBuildingID,BuildingType,PrimaryPropertyType,PropertyName,Address,ZipCode,TaxParcelIdentificationNumber,CouncilDistrictCode,Neighborhood,YearBuilt,...,SiteEnergyUse(kBtu),SiteEnergyUseWN(kBtu),SteamUse(kBtu),Electricity(kBtu),NaturalGas(kBtu),DefaultData,ComplianceStatus,Outlier,TotalGHGEmissions,GHGEmissionsIntensity
22,28,SPS-District K-12,K-12 School,Graham Hill Elementary,5149 S. Graham St.,98118.0,1102000138,2,SOUTHEAST,1961,...,1.539478e+06,1.713624e+06,0.0,9.369550e+05,6.025230e+05,True,Error - Correct Default Data,NaN,38.53,0.69
30,36,SPS-District K-12,K-12 School,Jane Addams Middle,11051 34th Ave. N.E.,98125.0,0752000170,5,NORTH,1949,...,6.779173e+06,7.800350e+06,0.0,1.947601e+06,4.831572e+06,True,Error - Correct Default Data,NaN,270.18,1.68
38,51,SPS-District K-12,K-12 School,Eckstein Middle,3003 N.E. 75th St.,98115.0,0425049022,4,NORTHEAST,1950,...,6.399128e+06,7.329818e+06,0.0,1.599684e+06,4.799443e+06,True,Error - Correct Default Data,NaN,266.05,1.52
86,125,SPS-District K-12,K-12 School,Mercer International Middle,1600 Columbian Way S.,98108.0,1624049214,2,GREATER DUWAMISH,1957,...,6.456026e+06,7.300978e+06,0.0,2.262820e+06,4.193207e+06,True,Error - Correct Default Data,NaN,238.48,1.82
87,127,SPS-District K-12,K-12 School,Orca K-8 School,5215 46th Ave. S.,98118.0,1704900242,2,SOUTHEAST,1989,...,1.633399e+06,1.749171e+06,0.0,1.436416e+06,1.969830e+05,True,Error - Correct Default Data,NaN,20.48,0.34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3036,27955,SPS-District K-12,K-12 School,Montlake Elementary,2409 22nd Ave. E.,98112.0,6788200735,3,EAST,1925,...,1.258619e+06,1.436271e+06,0.0,4.365350e+05,8.220840e+05,True,Error - Correct Default Data,NaN,46.70,1.71
3126,39332,SPS-District K-12,K-12 School,South Shore K-8,4800 S. Henderson,98118.0,2123700250,2,SOUTHEAST,2009,...,4.959298e+06,5.344758e+06,0.0,3.004396e+06,1.954902e+06,True,Error - Correct Default Data,NaN,124.77,0.90
3366,50210,Nonresidential COS,Office,Central West HQ / Brown Bear,1403 w howe,NaN,2425039137,7,MAGNOLIA / QUEEN ANNE,1952,...,5.026677e+05,5.585251e+05,0.0,5.026678e+05,0.000000e+00,True,Error - Correct Default Data,NaN,3.50,0.26
3369,50220,Nonresidential COS,Office,Genesee/SC SE HQ,4420 S Genesee,NaN,4154300585,2,SOUTHEAST,1960,...,3.878100e+05,4.141724e+05,0.0,2.775369e+05,1.102730e+05,True,Error - Correct Default Data,NaN,7.79,0.51


Ces bâtiment ont beaucoup de données. On va donc les garder.

In [25]:
building_consumption_clean[building_consumption_clean["ComplianceStatus"] == "Missing Data"]

,OSEBuildingID,BuildingType,PrimaryPropertyType,PropertyName,Address,ZipCode,TaxParcelIdentificationNumber,CouncilDistrictCode,Neighborhood,YearBuilt,...,SiteEnergyUse(kBtu),SiteEnergyUseWN(kBtu),SteamUse(kBtu),Electricity(kBtu),NaturalGas(kBtu),DefaultData,ComplianceStatus,Outlier,TotalGHGEmissions,GHGEmissionsIntensity
28,34,SPS-District K-12,K-12 School,Meany Building,301/300 21st Ave. E.,98112.0,0688000090,3,Central,1955,...,0.0,0.0,0.0,0.0,0.0,False,Missing Data,NaN,0.00,0.00
31,37,SPS-District K-12,K-12 School,John Hay Elementary,201 Garfield St.,98109.0,0809000005,7,MAGNOLIA / QUEEN ANNE,1989,...,0.0,0.0,0.0,1496816.0,0.0,False,Missing Data,NaN,10.43,0.20
62,87,SPS-District K-12,K-12 School,Arbor Heights Elementary,3701 S.W. 104th St.,98146.0,2518400005,1,SOUTHWEST,1948,...,0.0,0.0,0.0,600813.0,0.0,False,Missing Data,NaN,4.19,0.08
81,118,SPS-District K-12,K-12 School,Pathfinder K-8,1901 SW Genesee St.,98106.0,1324039116,1,DELRIDGE,1999,...,0.0,0.0,0.0,1698176.0,0.0,False,Missing Data,NaN,11.84,0.16
85,122,SPS-District K-12,K-12 School,John Muir Elementary,3301 S. Horton St.,98144.0,1524049002,2,SOUTHEAST,1991,...,0.0,0.0,0.0,2347138.0,0.0,False,Missing Data,NaN,16.36,0.28
95,140,SPS-District K-12,K-12 School,B.F. Day Elementary,3921 Linden Ave. N.,98103.0,1931301145,6,LAKE UNION,1991,...,0.0,0.0,0.0,2104286.0,0.0,False,Missing Data,NaN,14.67,0.22
133,217,SPS-District K-12,K-12 School,Whitman Middle,9201 15th Ave. NW.,98117.0,3526039131,6,BALLARD,1959,...,0.0,0.0,0.0,0.0,4318917.0,False,Missing Data,NaN,229.38,1.43
139,227,SPS-District K-12,K-12 School,Washington Middle,2101 S. Jackson,98144.0,3646100540,3,CENTRAL,1963,...,0.0,0.0,0.0,0.0,3217877.0,False,Missing Data,NaN,170.90,1.25
152,242,SPS-District K-12,K-12 School,Olympic View Elementary,504 N.E. 95th St.,98115.0,5100404840,5,NORTH,1989,...,0.0,0.0,0.0,0.0,0.0,False,Missing Data,NaN,0.00,0.00
614,820,SPS-District K-12,K-12 School,Bailey Gatzert Elementary,1301 E. Yesler Way,98122.0,0007600137,3,CENTRAL,1988,...,0.0,0.0,0.0,1957268.0,0.0,False,Missing Data,NaN,13.64,0.26


On peut supprimer ces lignes qui ont trop de valeurs manquantes.

In [26]:
building_consumption_clean = building_consumption_clean[
    ~building_consumption_clean["ComplianceStatus"].isin(["Missing Data"])
]

Outlier : Valeurs extrêmes identifiées

In [29]:
building_consumption_clean["Outlier"].value_counts()

Outlier
Low outlier     15
High outlier     2
Name: count, dtype: int64

Garder les valeurs qui prennent en compte la météo

In [32]:
building_consumption_clean = building_consumption_clean.drop(["SiteEUI(kBtu/sf)", "SourceEUI(kBtu/sf)"], axis=1)

In [33]:
building_consumption_clean["SiteEUIWN(kBtu/sf)"].min()

np.float64(0.0)

In [34]:
building_consumption_clean.loc[building_consumption_clean["SiteEUIWN(kBtu/sf)"].idxmin()]


OSEBuildingID                                         343
BuildingType                               NonResidential
PrimaryPropertyType                          Large Office
PropertyName                              Norton Building
Address                            801 2nd Ave Suite 1300
ZipCode                                           98104.0
TaxParcelIdentificationNumber                  0939000355
CouncilDistrictCode                                     7
Neighborhood                                     DOWNTOWN
YearBuilt                                            1958
NumberofBuildings                                     1.0
NumberofFloors                                         17
PropertyGFATotal                                   378525
PropertyGFAParking                                  84600
PropertyGFABuilding(s)                             293925
ListOfAllPropertyUseTypes                 Office, Parking
LargestPropertyUseType                             Office
LargestPropert

Ce bâtiment n'a pas de données concernant SourceEUIWN(kBtu/sf) , mais a des données en terme de SteamUse(kBtu) et Electricity(kBtu) , on va donc le garder.

In [35]:
building_consumption_clean["SiteEUIWN(kBtu/sf)"].mean()

np.float64(75.2008480047341)

In [36]:
building_consumption_clean["SiteEUIWN(kBtu/sf)"].max()

np.float64(834.4000244)

In [37]:
building_consumption_clean.loc[building_consumption_clean["SiteEUIWN(kBtu/sf)"].idxmax()]

OSEBuildingID                                       365
BuildingType                             NonResidential
PrimaryPropertyType                               Other
PropertyName                       UW Tower Data Center
Address                            4333 BROOKLYN AVE NE
ZipCode                                         98185.0
TaxParcelIdentificationNumber                1142000290
CouncilDistrictCode                                   4
Neighborhood                                  NORTHEAST
YearBuilt                                          1969
NumberofBuildings                                   1.0
NumberofFloors                                        5
PropertyGFATotal                                  36000
PropertyGFAParking                                    0
PropertyGFABuilding(s)                            36000
ListOfAllPropertyUseTypes                   Data Center
LargestPropertyUseType                      Data Center
LargestPropertyUseTypeGFA                       

Ok pour le max, c'est un DataCenter

In [38]:
building_consumption_clean.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1654 entries, 0 to 3375
Data columns (total 35 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   OSEBuildingID                    1654 non-null   int64  
 1   BuildingType                     1654 non-null   object 
 2   PrimaryPropertyType              1654 non-null   object 
 3   PropertyName                     1654 non-null   object 
 4   Address                          1654 non-null   object 
 5   ZipCode                          1638 non-null   float64
 6   TaxParcelIdentificationNumber    1654 non-null   object 
 7   CouncilDistrictCode              1654 non-null   int64  
 8   Neighborhood                     1654 non-null   object 
 9   YearBuilt                        1654 non-null   int64  
 10  NumberofBuildings                1652 non-null   float64
 11  NumberofFloors                   1654 non-null   int64  
 12  PropertyGFATotal         

In [39]:
building_consumption_clean.shape

(1654, 35)

#### TERMINER L'ANALYSE EXPLORATOIRE 

A réaliser : 
- Une analyse descriptive des données, y compris une explication du sens des colonnes gardées, des arguments derrière la suppression de lignes ou de colonnes, des statistiques descriptives et des visualisations pertinentes.

# Modélisation 

### Import des modules 

In [40]:
#Selection
from sklearn.model_selection import (
    train_test_split,
    GridSearchCV, 
    cross_validate,
)
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error 
from sklearn.inspection import permutation_importance

#Preprocess
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler

#Modèles
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor


ModuleNotFoundError: No module named 'sklearn'

### Feature Engineering

A réaliser : Enrichir le jeu de données actuel avec de nouvelles features issues de celles existantes. 

In [ ]:
# CODE FEATURE ENGINEERING

### Préparation des features pour la modélisation

A réaliser :
* Si ce n'est pas déjà fait, supprimer toutes les colonnes peu pertinentes pour la modélisation.
* Tracer la distribution de la cible pour vous familiariser avec l'ordre de grandeur. En cas d'outliers, mettez en place une démarche pour les supprimer.
* Débarrassez-vous des features redondantes en utilisant une matrice de corrélation.
* Réalisez différents graphiques pour comprendre le lien entre vos features et la target (boxplots, scatterplots, pairplot si votre nombre de features numériques n'est pas très élevé).
*  Séparez votre jeu de données en un Pandas DataFrame X (ensemble de feautures) et Pandas Series y (votre target).
* Si vous avez des features catégorielles, il faut les encoder pour que votre modèle fonctionne.

In [ ]:
# CODE PREPARATION DES FEATURES

### Comparaison de différents modèles supervisés

A réaliser :
* Pour chaque algorithme que vous allez tester, vous devez :
    * Réaliser au préalable une séparation en jeu d'apprentissage et jeu de test via une validation croisée.
    * Si les features quantitatives que vous souhaitez utiliser ont des ordres de grandeur très différents les uns des autres, et que vous utilisez un algorithme de regression qui est sensible à cette différence, alors il faut réaliser un scaling (normalisation) de la donnée au préalable.
    * Entrainer le modèle sur le jeu de Train
    * Prédire la cible sur la donnée de test (nous appelons cette étape, l'inférence).
    * Calculer les métriques de performance R2, MAE et RMSE sur le jeu de train et de test.
    * Interpréter les résultats pour juger de la fiabilité de l'algorithme.
* Vous pouvez choisir par exemple de tester un modèle linéaire, un modèle à base d'arbres et un modèle de type SVM
* Déterminer le modèle le plus performant parmi ceux testés.

In [ ]:
# CODE COMPARAISON DES MODELES

### Optimisation et interprétation du modèle

A réaliser :
* Reprennez le meilleur algorithme que vous avez sécurisé via l'étape précédente, et réalisez une GridSearch de petite taille sur au moins 3 hyperparamètres.
* Si le meilleur modèle fait partie de la famille des modèles à arbres (RandomForest, GradientBoosting) alors utilisez la fonctionnalité feature importance pour identifier les features les plus impactantes sur la performance du modèle. Sinon, utilisez la méthode Permutation Importance de sklearn. 

In [ ]:
# CODE OPTIMISATION ET INTERPRETATION DU MODELE